# Task 1 - Elementwise Operations and Broadcasting


PyTorch supports elementwise arithmetic on tensors of the same shape, and broadcasting for certain mismatched shapes (expanding dimensions of size 1 to match). Below are examples of valid and invalid broadcasting in elementwise operations:

Valid Broadcasting Example: Here a tensor of shape (3,4) is added to a tensor of shape (4,) (which is treated as (1,4)), broadcasting the smaller tensor across the first dimension:

In [ ]:
import torch
x = torch.rand(3, 4)        # shape (3,4)
y = torch.rand(4)           # shape (4,) -> treated as (1,4) for broadcasting
result = x + y              # y is broadcast along the 1st dim
print(result.shape)         # Output: torch.Size([3, 4])

Invalid Broadcasting Example: In this case, the shapes (3,4) and (2,) cannot be broadcast because the dimensions are incompatible (4 vs 2 in the last axis). 
This will raise a runtime error:

In [ ]:
a = torch.rand(3, 4)        # shape (3,4)
b = torch.rand(2)           # shape (2,) -> would be (1,2) for broadcasting
try:
    c = a + b               # ERROR: 4 (from a) vs 2 (from b) are not compatible
except RuntimeError as e:
    print(f"Broadcast error: {e}")


# Output: Broadcast error: The size of tensor a (4) must match the size of tensor b (2) at non-singleton dimension 1 (shows that the second dimension mismatch causes the error).

# Task 2 - Matrix Multiplication and Linear Layer Construction


In [ ]:
a = torch.rand(3, 4)        # shape (3,4)
b = torch.rand(2)           # shape (2,) -> would be (1,2) for broadcasting
try:
    c = a + b               # ERROR: 4 (from a) vs 2 (from b) are not compatible
except RuntimeError as e:
    print(f"Broadcast error: {e}")

In [ ]:
A = torch.randn(2, 3)
D = torch.randn(4, 2)
try:
    E = A @ D   # ERROR: A is 2x3, D is 4x2 (inner dims 3 vs 4 do not match)
except RuntimeError as e:
    print(f"Matmul error: {e}")

In [ ]:
import torch.nn as nn
linear = nn.Linear(in_features=5, out_features=2)  # expects input of shape (*,5)
x = torch.randn(4, 5)       # batch of 4, each with 5 features
y = linear(x) 
print(y.shape)             # Output: torch.Size([4, 2]) since out_features=2

In [ ]:
x_bad = torch.randn(4, 3)   # wrong feature dimension (should be 5)
try:
    y_bad = linear(x_bad)   # ERROR: expected input of size 5
except RuntimeError as e:
    print(f\"Linear layer error: {e}\")

# Task 3 - Reshaping and Flattening

Reshaping (or flattening) a tensor rearranges its dimensions without changing the total number of elements. A valid reshape must preserve the number of elements, whereas an invalid reshape (mismatched number of elements) will error:

Valid Reshape/Flatten Examples: Using .view or .reshape can flatten or change dimensions as long as the total count remains constant.

In [ ]:
t = torch.randn(2, 3, 4)      # shape (2,3,4), total elements = 24
flat = t.view(-1)             # flatten to 1D (24 elements)
print(flat.shape)            # Output: torch.Size([24])
reshaped = t.reshape(4, 6)    # reshape 24 elements to (4,6)
print(reshaped.shape)        # Output: torch.Size([4, 6])

# Here t.view(-1) flattens the tensor (PyTorch infers the size 24), and t.reshape(4,6) is valid because 4×6 = 24 matches the number of elements.

In [ ]:
# Invalid Reshape Example: Attempting to reshape to a size that doesn’t multiply to the same number of elements will throw an error:
try:
    wrong = t.view(5, 5)   # ERROR: 5x5 = 25 which does not equal 24
except RuntimeError as e:
    print(f"Reshape error: {e}")

# Task 4 - Convolutional Pipeline Construction

Below is a small CNN model with two convolutional layers, pooling, and a linear classifier. Shape transitions for each layer are annotated in comments. We also demonstrate an error case where a convolution’s input channels don’t match the data.



In [ ]:
import torch
import torch.nn as nn

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=8, kernel_size=3, padding=1)  
        # conv1: (N,1,28,28) -> (N,8,28,28)  [8 filters, 3x3 conv]
        self.pool  = nn.MaxPool2d(2)  
        # pool: (N,8,28,28) -> (N,8,14,14)   [2x2 pooling halves H,W]
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3, padding=1) 
        # conv2: (N,8,14,14) -> (N,16,14,14)
        self.pool2 = nn.MaxPool2d(2) 
        # pool2: (N,16,14,14) -> (N,16,7,7)
        self.fc    = nn.Linear(in_features=16*7*7, out_features=10) 
        # fc: (N,16*7*7) -> (N,10)
    def forward(self, x):
        x = self.conv1(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = self.pool2(x)
        x = torch.flatten(x, 1)          # flatten to shape (N, 16*7*7)
        x = self.fc(x)
        return x

model = SimpleCNN()
batch = torch.randn(1, 1, 28, 28)        # one sample, shape (1,28,28) with batch dim -> (N=1, C=1, H=28, W=28)
output = model(batch)
print(output.shape)  # Output: torch.Size([1, 10])

Shape explanation: An input of shape (1,1,28,28) goes through conv1 to (1,8,28,28), then pooling to (1,8,14,14). The second conv gives (1,16,14,14), pooling to (1,16,7,7). After flattening, it becomes (1, 16*7*7) = (1, 784), and the linear layer produces (1, 10).


In [ ]:
# Convolution Input Channel Mismatch (Error): If a conv layer is defined with an incorrect number of in_channels for the given input, a runtime error occurs. For example:

conv = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=3)  # expects 3 input channels
x = torch.randn(1, 1, 10, 10)  # actual data has 1 channel
try:
    y = conv(x)   # ERROR: conv expects 3 channels but got 1
except RuntimeError as e:
    print(f"Conv error: {e}")


# Output: Conv error: Given groups=1, weight of size [6, 3, 3, 3], expected input[1, 1, 10, 10] to have 3 channels, but got 1 channels instead. This clearly indicates the conv filter was configured for 3 input channels, but the input only had 1.

# 5 - Indexing, Slicing, and Batch Handling


In [ ]:
# Correct Indexing and Slicing Examples:
X = torch.randn(5, 3, 32, 32)    # shape (N=5, C=3, H=32, W=32)
img0 = X[0]                      # index first batch element, shape (3, 32, 32)
img0_batch = img0.unsqueeze(0)   # add batch dim back, shape (1, 3, 32, 32)
# Alternatively, X[0:1] would also give shape (1,3,32,32)
last_two = X[-2:]                # slice last 2 images, shape (2, 3, 32, 32)
first_channel = X[:, 0, :, :]    # slice first channel of each image, shape (5, 32, 32)
print(img0.shape, img0_batch.shape, last_two.shape, first_channel.shape)
# Output: torch.Size([3, 32, 32]) torch.Size([1, 3, 32, 32]) torch.Size([2, 3, 32, 32]) torch.Size([5, 32, 32])

# Here X[0] removes the batch dimension (producing a single image tensor), and unsqueeze(0) adds a new batch dimension. Slicing with -2: retrieves the last two batch elements. Indexing X[:, 0, :, :] extracts only the first channel from each image (reducing the channel dimension to size 1).


In [ ]:
# out of range index
try:
    X[5]        # ERROR: index 5 is out of range for a batch of size 5 (valid indices 0-4)
except IndexError as e:
    print(f"Indexing error: {e}")

In [ ]:
# Missing Batch Dimension: If a model expects a batch dimension but a single sample tensor is passed without one, it will error. For example, using the SimpleCNN defined above:

single_image = torch.randn(1, 28, 28)        # shape (C=1, H=28, W=28) with no batch dim
try:
    model(single_image)   # model expects shape (N,1,28,28), but got (1,28,28)
except RuntimeError as e:
    print(f"Batch dim error: {e}")
# Correct approach:
single_image_batch = single_image.unsqueeze(0)   # now shape (1, 1, 28, 28)
output = model(single_image_batch)               # works, output shape (1,10)

# The error would indicate a mismatch in number of dimensions. By adding an extra dimension (using unsqueeze(0)), the single image becomes a batch of size 1, resolving the issue.